In [1]:
import requests
import json
import pandas
import urllib.request
from bs4 import BeautifulSoup


In [2]:
pip install xlrd && pip install bs4 && pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
COMPANIES_SPREADSHEET_PWD = '/Users/aakash/Desktop/sellscale-api/notebooks/playground-with-websites.csv'

companies_df = pandas.read_csv(COMPANIES_SPREADSHEET_PWD)
len(companies_df)

8300

In [17]:

def find_points_from_website(url):
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()

    mystr = mybytes.decode("utf8")
    fp.close()

    c = []

    soup = BeautifulSoup(mystr, 'html.parser')
    ps = soup.find_all('p')
    for p in ps:
        if len(p.contents) > 0:
            contents = "".join([str(x) for x in p.contents])
            if '<' not in contents and len(contents) > 130 and 'contact us' not in contents.lower() and 'learn more' not in contents.lower() and 'disclaimer' not in contents.lower():
                contents = contents.replace(u'\xa0', u' ')
                contents = contents.strip()
                c.append(contents)
    
    return c


In [4]:

url = "https://palmbeachpreschool.com/"

def contains_procare(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    request=urllib.request.Request(url,None,headers)
    fp = urllib.request.urlopen(request)

    mybytes = fp.read()

    mystr = mybytes.decode("utf8")
    fp.close()

    c = []

    soup = BeautifulSoup(mystr, 'html.parser')
    return 'myprocare' in mystr

In [29]:
c = 0
for index, row in companies_df.iterrows():
    if c <= 264:
        c += 1
        continue
    website = row['Top Google Website']
    contains = False
    try:
        contains = contains_procare(website)
    except:
        pass
    with open('test.csv', 'a') as f:
        f.write(''.join([str(c), ',', str(website), ',', str(contains)]))
        f.write("\n")
        f.close()
    c += 1

In [8]:
for index, row in companies_df.iterrows():
    website = row['Top Google Website']
    try:
        if contains_procare(website):
            print(website, ' is using Procare')
    except:
        pass